## 特征选择(划分依据)

### 信息增益
+ 信息熵

假设一个样本集合D中有属于多个类别的样本，则当前集合的信息熵，即表示混乱程度：
$$entropy = ent(D) = -\sum_ip_ilogp_i,\mbox{每一个i表示一个类}$$

它的值越小，表示纯度越高。

+ 信息增益:用某一个特征进行分裂后纯度的提升

按照某一个特征进行划分后，信息熵减少的程度;

    信息增益 = D的信息熵 - (D|x)的信息熵
    
其中D|x在x这个变量下分裂后的条件信息熵为每一个子节点的加权平均(按照样本比例)，则信息增益为：

$$Gain(D,a) = Ent(D)-\sum_{v=1}^V\frac{D^v}{D}Ent(D^v)$$

+ 例子：ID3

### 信息增益比

+ 信息增益的缺点： 倾向于选择取数数目较多的属性。

倘若使用编号进行分裂，每一个节点，一个样本，信息增益肯定非常大，但是这个决策树不具有泛化能力。

+ 信息增益比
$$Gain_{ratio}(D,a)=\frac{Gain(D,a)}{IV(a)}$$
其中$$IV(a)=-\sum_{v=1}^V\frac{|D^v|}{|D|}log\frac{|D^v|}{|D|}$$
称为属性a的固有值，a的可能取值越多，则IV(a)的值通常会越大。

+ 缺点：增益率对取数数目较少的属性有所偏好。

+ 例子：C4.5(非直接用信息增益率，用了一个启发式的方法，先找到信息增益高于平均水平的属性，再从中选择增益率最高的)

### 基尼指数

CART中的划分规则

$$Gini(D)=1-\sum_{k=1}^{|y|}p_k^2$$
Gini系数反应了类别标记不一致的概率，因此越小表示数据集D的纯度越高。
属性a的基尼指数定义为：
$$Gini(D,a)=\sum_{v=1}^V\frac{|D^v|}{|D|}Gini(D^v)$$

每次选取使得gini指数最小的，作为最优划分属性。

## 决策树的生成(深度优先)
流程：

```
输入：训练数据D，特征集A，阈值e
输出：决策树T

1. if D中所有样本属于同一类别C then 将node标记为C类叶节点, return T
2. if A是空集，或者D中样本在A上取值都相同，则node标记为叶节点，类别标记为D中样本数最多的类，return T
3. 否则，选择最优划分属性a
4.如果信息增益少于阈值e，置其为单节点；否则，按照属性a进行划分
5.剔除属性a之后，对子树递归调用1-4
```

## 决策树剪枝

+ 预剪枝：在生成过程中，每次进行评价看是否能带来泛化性能的提升，不行就停止划分并将当前结点标记为叶节点。
  + 缺点：当前分裂虽然可能导致泛化能力下降，但之后的后续划分可能导致性能显著提高。
+ 后剪枝：生成完后再剪枝，向上搜索看以某一个内部结点为叶节点的子树是不是有更好的泛化能力。
  + 优点：欠拟合风险较小
  + 缺点：在生成完之后再进行，时间等开销比较大
  
### 损失函数
设树的叶结点个数为|T|,t是树T的叶结点，该叶结点有$N_t$个样本点，其中k类的样本点有$N_{tk}$，$H_t(T)$为叶节点t上的经验熵
$$C_{\alpha}(T)=\sum_{t=1}^{|T|}N_tH_t(T)+\alpha|T|$$

从叶结点出发，往上面搜寻，若剪枝能带来损失函数的降低，则剪枝。

### CART剪枝

+ 剪枝，形成一个子树序列
从整体树$T_0$进行剪枝，对$T_0$的任意内部结点t，以t为单节点的损失函数为:
$$C_{\alpha}(t)=C(t)+\alpha$$
以t为根节点的子树$T_t$的损失函数为:
$$C_{\alpha}(T_t)=C(T_t)+\alpha|T_t|$$

随着 $\alpha$的增大，临界点处，$T_t$与t的损失函数相当，因此需要对$T_t$进行剪枝


for t in $T_0$的所有内部结点:

   计算 $g(t)=\frac{C(t)-C(T_t)}{|T_t|-1}$
求出$g(t)$最小，即最小的一个$\alpha$，$T_0$减去最小的$T_t$，得到的树变为$T_1$，$\alpha_1=g(t)$;$T_1$是区间$[\alpha_1,\alpha_2)$的最优子树。

+ 从这些子树序列中，交叉验证得到最优子树$T_{\alpha}$

上面产生的子树序列，交叉验证得到最优的子树。


## 连续值(离散化)
如C4.5一样，使用二分法将其离散化,由于$[a_i,a_{i+1}]$区间内的任何一个划分点不影响结果，因此划分点一般是两个样本点的中位点。

选取最优的划分点(训练样本两个值的中点)，最优的属性

## 缺失值处理
见周志华(86)

## 多变量决策树
特征对应于一个维度，多个特征其实就决定了高维空间中的分类边界，值得注意的是，这些分类边界都是轴平行的。如何使得分类边界是曲线的呢？

多变量决策树，也称作斜划分。每个非叶结点，都是形如 $\sum_i w_ia_i=t$的线性分类器

In [22]:
def entropy(y):
    """
    求信息熵
    Parameters
    ----------
    y: ndarray,表示该样本集合的label
    """
    import numpy as np
    counts = np.unique(y,return_counts=True)[1]
    n = np.float32(len(y))
    
    return np.sum(-counts/n * np.log(counts/n))
    
    

In [1]:
import numpy as np

In [9]:
(np.unique([1,2,1,2,3,1,2],return_counts=True))[1]

array([0, 0, 0])

In [11]:
tmp=(np.unique([1,2,1,2,3,1,2],return_counts=True))[1]

In [13]:
tmp/7.0

array([ 0.42857143,  0.42857143,  0.14285714])

In [19]:
list(tmp).map?

In [23]:
entropy([1,2,2,1,3,2,1])

1.0042424730540764